# Model
Exicting part, try different types regression model.


In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectFromModel

from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

import pickle

import warnings
warnings.filterwarnings('ignore')

In [2]:
#read data, specify index so it's easier to join and search using loc
path = '../../data/new-york-city-airbnb-open-data/'
listings_csv = os.path.join(path,'model_columns_listings.csv')
listings = pd.read_csv(listings_csv)

In [3]:
#shape
listings.shape

(25604, 252)

In [4]:
listings.columns.values

array(['Unnamed: 0', 'id', 'host_since', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_listings_count', 'host_identity_verified', 'latitude',
       'longitude', 'is_location_exact', 'accommodates', 'bathrooms',
       'bedrooms', 'beds', 'price', 'security_deposit', 'cleaning_fee',
       'guests_included', 'extra_people', 'number_of_reviews',
       'number_of_reviews_ltm', 'first_review', 'last_review',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'instant_bookable', 'reviews_per_month',
       'last_review_days_ago', 'host_verifications_ facebook',
       'host_verifications_ google', 'host_verifications_ government_id',
       'host_verifications_ identity_manual', 'host_verifications_ jumio',
       'host_verifications_ kba', 'host_verifications_ manual_

In [5]:
#head
listings.head(10)

,Unnamed: 0,id,host_since,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_identity_verified,latitude,...,cancellation_policy_moderate,cancellation_policy_strict,cancellation_policy_strict_14_with_grace_period,cancellation_policy_super_strict_30,cancellation_policy_super_strict_60,bed_type_Airbed,bed_type_Couch,bed_type_Futon,bed_type_Pull-out Sofa,bed_type_Real Bed
0,1,2595,733294,3.0,87%,38%,0,6.0,1,40.75362,...,0,0,1,0,0,0,0,0,0,1
1,2,3831,733383,1.0,83%,96%,0,1.0,1,40.68514,...,1,0,0,0,0,0,0,0,0,1
2,3,5099,733440,2.5,NaN,71%,0,1.0,0,40.74767,...,1,0,0,0,0,0,0,0,0,1
3,5,5121,733441,2.5,NaN,67%,0,1.0,0,40.68688,...,0,0,1,0,0,0,0,1,0,0
4,6,5178,733469,3.0,90%,100%,0,1.0,0,40.76489,...,0,0,1,0,0,0,0,0,0,1
5,9,5441,733454,2.0,100%,56%,1,1.0,1,40.76160,...,0,0,0,0,0,0,0,0,0,1
6,11,5803,733476,2.0,100%,99%,1,3.0,1,40.66829,...,0,0,1,0,0,0,0,0,0,1
7,12,6021,733493,3.0,90%,84%,0,2.0,0,40.79826,...,0,0,1,0,0,0,0,0,0,1
8,14,6848,733533,1.0,90%,96%,0,1.0,0,40.70837,...,0,0,1,0,0,0,0,0,0,1
9,15,6990,733539,1.0,100%,89%,1,3.0,0,40.78962,...,1,0,0,0,0,0,0,0,0,1


# Features/Target
Features to be included and test/train split, take out 'latitude', 'longitude'
Also due to removing certain price, the following is gone: 'property_type_Train', 'property_type_Dome house', 'property_type_Dorm', 'host_verifications_google', 'property_type_Island', 'property_type_In-law', 'property_type_Yurt', 'property_type_Bus', 'property_type_Timeshare', 'property_type_Farm stay', 'property_type_Treehouse'

In [6]:
# X = listings[['host_since', 'host_response_time', 'host_is_superhost', 'host_listings_count', 'host_identity_verified', 'is_location_exact', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'security_deposit', 'cleaning_fee', 'guests_included','extra_people', 'number_of_reviews', 'number_of_reviews_ltm', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'instant_bookable', 'reviews_per_month', 'host_verifications_ facebook', 'host_verifications_ google', 'host_verifications_ government_id', 'host_verifications_ identity_manual', 'host_verifications_ jumio', 'host_verifications_ kba', 'host_verifications_ manual_offline', 'host_verifications_ manual_online', 'host_verifications_ offline_government_id', 'host_verifications_ phone', 'host_verifications_ reviews', 'host_verifications_ selfie', 'host_verifications_ sent_id', 'host_verifications_ sesame', 'host_verifications_ sesame_offline', 'host_verifications_ weibo', 'host_verifications_ work_email', 'host_verifications_ zhima_selfie', 'host_verifications_email', 'host_verifications_facebook', 'host_verifications_google', 'host_verifications_jumio', 'host_verifications_offline_government_id', 'host_verifications_phone', 'host_verifications_reviews', 'neighbourhood_group_cleansed_Bronx', 'neighbourhood_group_cleansed_Brooklyn', 'neighbourhood_group_cleansed_Manhattan', 'neighbourhood_group_cleansed_Queens', 'neighbourhood_group_cleansed_Staten Island', 'property_type_Aparthotel', 'property_type_Apartment','property_type_Barn', 'property_type_Bed and breakfast', 'property_type_Boat', 'property_type_Boutique hotel', 'property_type_Bungalow', 'property_type_Bus', 'property_type_Cabin', 'property_type_Camper/RV', 'property_type_Castle', 'property_type_Cave', 'property_type_Condominium', 'property_type_Cottage', 'property_type_Dome house', 'property_type_Dorm', 'property_type_Earth house', 'property_type_Farm stay', 'property_type_Guest suite', 'property_type_Guesthouse', 'property_type_Hostel', 'property_type_Hotel', 'property_type_House', 'property_type_Houseboat', 'property_type_In-law', 'property_type_Island', 'property_type_Lighthouse','property_type_Loft', 'property_type_Other', 'property_type_Resort', 'property_type_Serviced apartment', 'property_type_Tent', 'property_type_Timeshare', 'property_type_Tiny house', 'property_type_Townhouse', 'property_type_Train', 'property_type_Treehouse', 'property_type_Villa', 'property_type_Yurt', 'room_type_Entire home/apt', 'room_type_Hotel room', 'room_type_Private room', 'room_type_Shared room', 'cancellation_policy_flexible', 'cancellation_policy_moderate', 'cancellation_policy_strict', 'cancellation_policy_strict_14_with_grace_period', 'cancellation_policy_super_strict_30', 'cancellation_policy_super_strict_60', 'bed_type_Airbed', 'bed_type_Couch', 'bed_type_Futon','bed_type_Pull-out Sofa', 'bed_type_Real Bed', 'amenities_ toilet', 'amenities_24-hour check-in', 'amenities_Accessible-height bed','amenities_Accessible-height toilet', 'amenities_Air conditioning', 'amenities_Air purifier', 'amenities_BBQ grill', 'amenities_Baby bath', 'amenities_Baby monitor', 'amenities_Babysitter recommendations', 'amenities_Baking sheet', 'amenities_Barbecue utensils', 'amenities_Bath towel', 'amenities_Bathrobes', 'amenities_Bathroom essentials', 'amenities_Bathtub', 'amenities_Bathtub with bath chair', 'amenities_Beach essentials', 'amenities_Beachfront', 'amenities_Bed linens', 'amenities_Bedroom comforts', 'amenities_Bluetooth speaker', 'amenities_Body soap', 'amenities_Bottled water', 'amenities_Bread maker', 'amenities_Breakfast','amenities_Building staff', 'amenities_Buzzer/wireless intercom', 'amenities_Cable TV',
# 'amenities_Carbon monoxide detector', 'amenities_Cat(s)', 'amenities_Changing table', 'amenities_Children’s books and toys', 'amenities_Children’s dinnerware',
# 'amenities_Cleaning before checkout', 'amenities_Coffee maker',
# 'amenities_Cooking basics', 'amenities_Crib', 'amenities_Disabled parking spot',
# 'amenities_Dishes and silverware', 'amenities_Dishwasher', 'amenities_Dog(s)', 'amenities_Doorman', 'amenities_Dryer', 'amenities_EV charger', 'amenities_Electric profiling bed', 'amenities_Elevator', 'amenities_Essentials', 'amenities_Ethernet connection', 'amenities_Extra pillows and blankets', 'amenities_Extra space around bed', 'amenities_Family/kid friendly', 'amenities_Fire extinguisher', 'amenities_Fireplace guards', 'amenities_Firm mattress', 'amenities_First aid kit', 'amenities_Fixed grab bars for shower', 'amenities_Fixed grab bars for toilet', 'amenities_Flat path to guest entrance',
# 'amenities_Free parking on premises', 'amenities_Free street parking', 'amenities_Full kitchen', 'amenities_Game console', 'amenities_Garden or backyard', 'amenities_Ground floor access', 'amenities_Gym', 'amenities_Hair dryer', 'amenities_Handheld shower head', 'amenities_Hangers', 'amenities_Heating', 'amenities_High chair', 'amenities_Host greets you', 'amenities_Hot tub',
# 'amenities_Hot water', 'amenities_Hot water kettle', 'amenities_Indoor fireplace', 'amenities_Internet', 'amenities_Iron', 'amenities_Keypad', 'amenities_Kitchen',
# 'amenities_Kitchenette', 'amenities_Lake access', 'amenities_Laptop friendly workspace',
# 'amenities_Lock on bedroom door', 'amenities_Lockbox', 'amenities_Long term stays allowed',
# 'amenities_Luggage dropoff allowed', 'amenities_Microwave', 'amenities_Mini bar', 'amenities_Mini fridge', 'amenities_Mobile hoist', 'amenities_No stairs or steps to enter', 'amenities_Other', 'amenities_Other pet(s)', 'amenities_Outlet covers', 'amenities_Oven', 'amenities_Pack ’n Play/travel crib', 'amenities_Paid parking off premises', 'amenities_Paid parking on premises', 'amenities_Patio or balcony', 'amenities_Pets allowed', 'amenities_Pets live on this property', 'amenities_Pocket wifi', 'amenities_Pool', 'amenities_Private bathroom', 'amenities_Private entrance', 'amenities_Private living room', 'amenities_Record player', 'amenities_Refrigerator', 'amenities_Room-darkening shades', 'amenities_Safe', 'amenities_Safety card', 'amenities_Self check-in', 'amenities_Shampoo', 'amenities_Shower chair', 'amenities_Shower gel',
# 'amenities_Single level home', 'amenities_Ski-in/Ski-out', 'amenities_Slippers', 'amenities_Smart lock', 'amenities_Smoke detector', 'amenities_Smoking allowed', 'amenities_Snacks', 'amenities_Stair gates', 'amenities_Step-free shower', 'amenities_Stove',
# 'amenities_Suitable for events', 'amenities_TV', 'amenities_Table corner guards', 'amenities_Toilet paper', 'amenities_Trash can', 'amenities_Turndown service',
# 'amenities_Washer', 'amenities_Washer/Dryer', 'amenities_Waterfront', 'amenities_Well-lit path to entrance', 'amenities_Wheelchair accessible', 'amenities_Wide clearance to shower',
# 'amenities_Wide doorway to guest bathroom', 'amenities_Wide entrance', 'amenities_Wide entrance for guests', 'amenities_Wide entryway', 'amenities_Wide hallways', 'amenities_Wifi', 'amenities_Window guards']]

X = listings[['host_since', 'host_response_time', 'host_is_superhost', 'host_listings_count', 'host_identity_verified', 'is_location_exact', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'security_deposit', 'cleaning_fee', 'guests_included','extra_people', 'number_of_reviews', 'number_of_reviews_ltm', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'instant_bookable', 'reviews_per_month', 'host_verifications_ facebook', 'host_verifications_ google', 'host_verifications_ government_id', 'host_verifications_ identity_manual', 'host_verifications_ jumio', 'host_verifications_ kba', 'host_verifications_ manual_offline', 'host_verifications_ manual_online', 'host_verifications_ offline_government_id', 'host_verifications_ phone', 'host_verifications_ reviews', 'host_verifications_ selfie', 'host_verifications_ sent_id', 'host_verifications_ sesame', 'host_verifications_ sesame_offline', 'host_verifications_ weibo', 'host_verifications_ work_email', 'host_verifications_ zhima_selfie', 'host_verifications_email', 'host_verifications_facebook', 'host_verifications_jumio', 'host_verifications_offline_government_id', 'host_verifications_phone', 'host_verifications_reviews', 'neighbourhood_group_cleansed_Bronx', 'neighbourhood_group_cleansed_Brooklyn', 'neighbourhood_group_cleansed_Manhattan', 'neighbourhood_group_cleansed_Queens', 'neighbourhood_group_cleansed_Staten Island', 'property_type_Apartment','property_type_Barn', 'property_type_Boat', 'property_type_Bungalow', 'property_type_Cabin', 'property_type_Camper/RV', 'property_type_Castle', 'property_type_Cave', 'property_type_Condominium', 'property_type_Cottage', 'property_type_Earth house', 'property_type_Guest suite', 'property_type_Guesthouse', 'property_type_Hostel', 'property_type_House', 'property_type_Houseboat', 'property_type_Lighthouse','property_type_Loft', 'property_type_Other', 'property_type_Serviced apartment', 'property_type_Tent', 'property_type_Tiny house', 'property_type_Townhouse', 'property_type_Villa', 'room_type_Entire home/apt', 'room_type_Hotel room', 'room_type_Private room', 'room_type_Shared room', 'cancellation_policy_flexible', 'cancellation_policy_moderate', 'cancellation_policy_strict', 'cancellation_policy_strict_14_with_grace_period', 'cancellation_policy_super_strict_30', 'cancellation_policy_super_strict_60', 'bed_type_Airbed', 'bed_type_Couch', 'bed_type_Futon','bed_type_Pull-out Sofa', 'bed_type_Real Bed', 'amenities_ toilet', 'amenities_24-hour check-in', 'amenities_Accessible-height bed','amenities_Accessible-height toilet', 'amenities_Air conditioning', 'amenities_Air purifier', 'amenities_BBQ grill', 'amenities_Baby bath', 'amenities_Baby monitor', 'amenities_Babysitter recommendations', 'amenities_Baking sheet', 'amenities_Barbecue utensils', 'amenities_Bath towel', 'amenities_Bathrobes', 'amenities_Bathroom essentials', 'amenities_Bathtub', 'amenities_Bathtub with bath chair', 'amenities_Beach essentials', 'amenities_Beachfront', 'amenities_Bed linens', 'amenities_Bedroom comforts', 'amenities_Bluetooth speaker', 'amenities_Body soap', 'amenities_Bottled water', 'amenities_Bread maker', 'amenities_Breakfast','amenities_Building staff', 'amenities_Buzzer/wireless intercom', 'amenities_Cable TV',
'amenities_Carbon monoxide detector', 'amenities_Cat(s)', 'amenities_Changing table', 'amenities_Children’s books and toys', 'amenities_Children’s dinnerware',
'amenities_Cleaning before checkout', 'amenities_Coffee maker',
'amenities_Cooking basics', 'amenities_Crib', 'amenities_Disabled parking spot',
'amenities_Dishes and silverware', 'amenities_Dishwasher', 'amenities_Dog(s)', 'amenities_Doorman', 'amenities_Dryer', 'amenities_EV charger', 'amenities_Electric profiling bed', 'amenities_Elevator', 'amenities_Essentials', 'amenities_Ethernet connection', 'amenities_Extra pillows and blankets', 'amenities_Extra space around bed', 'amenities_Family/kid friendly', 'amenities_Fire extinguisher', 'amenities_Fireplace guards', 'amenities_Firm mattress', 'amenities_First aid kit', 'amenities_Fixed grab bars for shower', 'amenities_Fixed grab bars for toilet', 'amenities_Flat path to guest entrance',
'amenities_Free parking on premises', 'amenities_Free street parking', 'amenities_Full kitchen', 'amenities_Game console', 'amenities_Garden or backyard', 'amenities_Ground floor access', 'amenities_Gym', 'amenities_Hair dryer', 'amenities_Handheld shower head', 'amenities_Hangers', 'amenities_Heating', 'amenities_High chair', 'amenities_Host greets you', 'amenities_Hot tub',
'amenities_Hot water', 'amenities_Hot water kettle', 'amenities_Indoor fireplace', 'amenities_Internet', 'amenities_Iron', 'amenities_Keypad', 'amenities_Kitchen',
'amenities_Kitchenette', 'amenities_Lake access', 'amenities_Laptop friendly workspace',
'amenities_Lock on bedroom door', 'amenities_Lockbox', 'amenities_Long term stays allowed',
'amenities_Luggage dropoff allowed', 'amenities_Microwave', 'amenities_Mini bar', 'amenities_Mini fridge', 'amenities_Mobile hoist', 'amenities_No stairs or steps to enter', 'amenities_Other', 'amenities_Other pet(s)', 'amenities_Outlet covers', 'amenities_Oven', 'amenities_Pack ’n Play/travel crib', 'amenities_Paid parking off premises', 'amenities_Paid parking on premises', 'amenities_Patio or balcony', 'amenities_Pets allowed', 'amenities_Pets live on this property', 'amenities_Pocket wifi', 'amenities_Pool', 'amenities_Private bathroom', 'amenities_Private entrance', 'amenities_Private living room', 'amenities_Record player', 'amenities_Refrigerator', 'amenities_Room-darkening shades', 'amenities_Safe', 'amenities_Safety card', 'amenities_Self check-in', 'amenities_Shampoo', 'amenities_Shower chair', 'amenities_Shower gel',
'amenities_Single level home', 'amenities_Ski-in/Ski-out', 'amenities_Slippers', 'amenities_Smart lock', 'amenities_Smoke detector', 'amenities_Smoking allowed', 'amenities_Snacks', 'amenities_Stair gates', 'amenities_Step-free shower', 'amenities_Stove',
'amenities_Suitable for events', 'amenities_TV', 'amenities_Table corner guards', 'amenities_Toilet paper', 'amenities_Trash can', 'amenities_Turndown service',
'amenities_Washer', 'amenities_Washer/Dryer', 'amenities_Waterfront', 'amenities_Well-lit path to entrance', 'amenities_Wheelchair accessible', 'amenities_Wide clearance to shower',
'amenities_Wide doorway to guest bathroom', 'amenities_Wide entrance', 'amenities_Wide entrance for guests', 'amenities_Wide entryway', 'amenities_Wide hallways', 'amenities_Wifi', 'amenities_Window guards', 'last_review_days_ago']]

y = listings['price']

# Split your data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Lasso Regression

In [7]:
#using CV
regressor = Lasso(alpha=0.5)
regressor = regressor.fit(X_train, y_train) 

#cross validation, 10 fold to try next
scores = cross_val_score(estimator=regressor, X=X_train, y=y_train, cv=10, scoring="neg_mean_squared_error")
rmse_scores = np.sqrt(-scores)

print(rmse_scores)


[ 81.47785242 100.13979121 100.57051061  81.25182391  89.49859447
  92.28007405  99.10364875  84.44522038  94.04554651  73.84775531]


In [8]:
pred_train = regressor.predict(X_train)
print(rmse_scores.mean())
print(r2_score(y_train, pred_train))


89.6660817623358
0.5200322183719387


In [9]:
pred_test= regressor.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,pred_test))) 
print(r2_score(y_test, pred_test))

89.78939003420378
0.5051723850437027


# Ridge Regression

In [10]:
#using CV
regressor = Ridge(alpha=0.5)
regressor = regressor.fit(X_train, y_train) 

#cross validation, 10 fold to try next
scores = cross_val_score(estimator=regressor, X=X_train, y=y_train, cv=10, scoring="neg_mean_squared_error")
rmse_scores = np.sqrt(-scores)
print(rmse_scores)


[81.76475108 98.73691514 99.24970706 81.53943714 89.38857227 91.01297806
 98.28373933 83.69099075 92.95140671 73.88751625]


In [11]:
pred_train = regressor.predict(X_train)
print(rmse_scores.mean())
print(r2_score(y_train, pred_train))


89.05060137979535
0.5348464231724124


In [12]:
pred_test= regressor.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,pred_test))) 
print(r2_score(y_test, pred_test))

89.76320486949167
0.5054609548459779


# Random Forest

In [13]:
#using CV
regressor = RandomForestRegressor(n_estimators=100, random_state=None, min_samples_split=100)
regressor = regressor.fit(X_train, y_train) 

#cross validation, 10 fold to try next
scores = cross_val_score(estimator=regressor, X=X_train, y=y_train, cv=10, scoring="neg_mean_squared_error")
rmse_scores = np.sqrt(-scores) 
print(rmse_scores)

[ 77.22757397  94.18288757  92.83789028  74.67241394  83.631813
  88.75348364 101.54862682  78.57109204  84.92787512  72.25292251]


In [14]:
pred_train= regressor.predict(X_train)
print(rmse_scores.mean())
print(r2_score(y_train, pred_train))

84.86065788770013
0.661863398764635


In [15]:
pred_test= regressor.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,pred_test))) 
print(r2_score(y_test, pred_test))

85.10968837213552
0.5554077514134815


# Feature Importance for Random Forest

In [16]:
#get features importance
features = regressor.feature_importances_

#get column names from our list of features
column_names=X.columns.values.tolist()

#match features importance to column name
df = pd.DataFrame([features], columns=column_names)


In [17]:
df.head()

,host_since,host_response_time,host_is_superhost,host_listings_count,host_identity_verified,is_location_exact,accommodates,bathrooms,bedrooms,beds,...,amenities_Wheelchair accessible,amenities_Wide clearance to shower,amenities_Wide doorway to guest bathroom,amenities_Wide entrance,amenities_Wide entrance for guests,amenities_Wide entryway,amenities_Wide hallways,amenities_Wifi,amenities_Window guards,last_review_days_ago
0,0.011922,0.000884,0.000275,0.003677,0.000131,0.000162,0.039046,0.232395,0.092476,0.002004,...,0.000409,0.0,0.0,0.0,0.000046,0.0,0.000054,0.000195,0.000041,0.004628


In [18]:
#features importance sorted
features_sorted = df.iloc[0].sort_values(ascending=False)

In [19]:
features_sorted.head(60)


room_type_Entire home/apt                 0.295014
bathrooms                                 0.232395
neighbourhood_group_cleansed_Manhattan    0.095101
bedrooms                                  0.092476
cleaning_fee                              0.091408
accommodates                              0.039046
security_deposit                          0.013052
host_since                                0.011922
property_type_Serviced apartment          0.009884
guests_included                           0.008157
property_type_Loft                        0.007312
extra_people                              0.006597
review_scores_location                    0.006191
reviews_per_month                         0.005273
last_review_days_ago                      0.004628
amenities_Free parking on premises        0.004169
number_of_reviews                         0.003988
host_listings_count                       0.003677
amenities_Dishwasher                      0.003638
amenities_Kitchen              

In [20]:
#features more than mean of importance
filt = features_sorted.values > features_sorted.mean()
features_sorted[filt].count()

15

In [29]:
X = listings[['room_type_Entire home/apt',
                'bathrooms',
                'neighbourhood_group_cleansed_Manhattan',
                'cleaning_fee',
                'bedrooms',
                'accommodates',
                'security_deposit',
                'host_since',
                'property_type_Serviced apartment',
                'guests_included',
                'property_type_Loft',
                'review_scores_location',
                'extra_people',
                'reviews_per_month',
                'last_review_days_ago',
                'amenities_Kitchen',
                'number_of_reviews',
                'host_listings_count',
                'amenities_Free parking on premises',
                'amenities_Dishwasher',
                'amenities_Essentials',
                'amenities_Patio or balcony',
                'number_of_reviews_ltm',
                'amenities_Indoor fireplace',
                'room_type_Hotel room',
                'review_scores_rating',
                'amenities_Hangers',
                'beds',
                'amenities_Building staff',
                'amenities_Free street parking',
                'review_scores_communication',
                'amenities_Baby monitor',
                'review_scores_cleanliness',
                'amenities_Doorman',
                'review_scores_checkin',
                'amenities_Washer',
                'neighbourhood_group_cleansed_Brooklyn',
                'amenities_Dryer',
                'property_type_Apartment',
                'amenities_Gym',
                'host_response_time',
                'amenities_Suitable for events',
                'amenities_Smoking allowed',
                'amenities_Elevator',
                'host_verifications_ selfie',
                'amenities_Shampoo',
                'amenities_Pocket wifi',
                'room_type_Shared room',
                'property_type_Condominium',
                'amenities_EV charger',
                'bed_type_Couch',
                'room_type_Private room',
                'amenities_Wheelchair accessible',
                'amenities_Hot tub',
                'host_is_superhost',
                'review_scores_value',
                'amenities_Pets allowed',
                'amenities_Pool',
                'property_type_Townhouse',
                'property_type_Guest suite'
]]

y = listings['price']


# Split your data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Random Forest Rerun

In [36]:
#rerun using featuers that are heavily used
regressor = RandomForestRegressor(n_estimators=100, random_state=None, min_samples_split=100)
regressor = regressor.fit(X_train, y_train) 

#cross validation, 10 fold to try next
scores = cross_val_score(estimator=regressor, X=X_train, y=y_train, cv=10, scoring="neg_mean_squared_error")
rmse_scores = np.sqrt(-scores)
print(rmse_scores)

[ 77.1667737   94.20656345  92.30358485  74.0532268   83.71534802
  87.7672232  101.56121488  77.93986158  86.35027464  71.92777152]


In [37]:
pred_train= regressor.predict(X_train)
print(rmse_scores.mean())
print(r2_score(y_train, pred_train))

84.69918426454215
0.6610946503102648


In [38]:
pred_test= regressor.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,pred_test))) 
print(r2_score(y_test, pred_test))

85.39840780556287
0.5523862353467356


# Ridge Regression Rerun

In [33]:
#using CV
regressor = Ridge(alpha=0.1)
regressor = regressor.fit(X_train, y_train) 

#cross validation, 10 fold to try next
scores = cross_val_score(estimator=regressor, X=X_train, y=y_train, cv=10, scoring="neg_mean_squared_error")
rmse_scores = np.sqrt(-scores)
print(rmse_scores)

[81.29306594 99.172288   99.48273291 80.97385739 89.82899053 91.54560266
 98.79149587 84.20511966 93.208001   74.15858428]


In [34]:
pred_train = regressor.predict(X_train)
print(rmse_scores.mean())
print(r2_score(y_train, pred_train))

89.26597382375762
0.5247798898753151


In [35]:
pred_test= regressor.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,pred_test))) 
print(r2_score(y_test, pred_test))

89.89947237627374
0.5039583179000422


# Model Output for Testing Flask
Using very few features

In [6]:
X = listings[['room_type_Entire home/apt',
                'bathrooms',
                'neighbourhood_group_cleansed_Manhattan',
                'cleaning_fee',
                'bedrooms',
                'accommodates',
                'security_deposit'
]]

y = listings['price']

# Split your data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Lasso Regression
For testing flask

In [7]:
#using CV
regressor = Lasso(alpha=0.1)
regressor = regressor.fit(X_train, y_train) 

#cross validation, 10 fold to try next
scores = cross_val_score(estimator=regressor, X=X_train, y=y_train, cv=10, scoring="neg_mean_squared_error")
rmse_scores = np.sqrt(-scores)

print(rmse_scores)

[ 84.764473   102.7244807  103.65396042  84.22046589  91.46900529
  95.40114448 102.11204679  87.93890438  96.8238047   77.05871149]


In [8]:
pred_train = regressor.predict(X_train)
print(rmse_scores.mean())
print(r2_score(y_train, pred_train))

92.61669971481928
0.48452290493972583


In [9]:
pred_test= regressor.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,pred_test))) 
print(r2_score(y_test, pred_test))

92.56442941204261
0.47411335419090317


In [10]:
# Saving model to disk
pickle.dump(regressor, open('model.pkl','wb'))